In [2]:
pip install mediapipe

  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/da/f9/393618415e43f76a694ce8d1c8147ed0ec5f05cecc6cab2e02dcb4a0bc3f/mediapipe-0.10.7-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=2.0 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/81/3c/bbb3ceee9fbefc505f98c24dafda68c7b3c4f83b6951c0712b4623fe4cce/opencv_contrib_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
     ---------------------------------------- 0.0/162.1 kB ? eta -:--:--
     ---------------------- ---------------- 92.2/162.1 kB 2

In [3]:
pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/80/6f/57d36f6507e432d7fc1956b2e9e8530c5c2d2bfcd8821bcbfae271cd6688/tensorflow-2.14.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.14.0 from https://files.pythonhosted.org/packages/ad/6e/1bfe367855dd87467564f7bf9fa14f3b17889988e79598bc37bf18f5ffb6/tensorflow_intel-2.14.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ---------------------------------------- 57.5/57.5 kB 1.5 MB/s eta 0:00:00
  Obtaining dependency information for libclang>=13.0.0 from https://files.pythonhosted.org/packages/02/8c/dc970bc00867fe290e8c8a7befa1635af716a9ebdfe3fb9dce0ca4b522ce/libclang-16.0.6-py2.py3-none-win_amd64.whl.metadata
  Obtaining dependency information for ml-dtypes==0.2.0 from https://files.pythonhosted.org/packages/08/89/c727fde1a3d12586e0b8c01abf53754707d76beaa9987640e70807d4545f/ml_dtypes-

In [4]:
pip install pyvirtualcam

     ---------------------------------------- 0.0/167.7 kB ? eta -:--:--
     -- ------------------------------------- 10.2/167.7 kB ? eta -:--:--
     ------------- ----------------------- 61.4/167.7 kB 648.1 kB/s eta 0:00:01
     ------------------------ ----------- 112.6/167.7 kB 726.2 kB/s eta 0:00:01
     -------------------------------------- 167.7/167.7 kB 1.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/38/d2/3e8c13ffc37ca5ebc6f382b242b44acb43eb489042e1728407ac3904e72f/opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.1 MB 1.4 MB/s eta 0:00:28
   ---------------------------------------- 0.1/38.1 MB 2.1 MB/s eta 0:00:18
   ---------------------------------------- 0.3/38.1 MB 2.5 MB/s eta 0:00:15
   ---------------------------------------- 0.5/38.1 MB 2.8 MB/s eta 0:00:14
    --------------------------------------- 0.7/38.1 MB 3.4 MB/s eta 0:00:12
    --------------------------------------- 0.9/38.1 MB 3.3 MB/s eta 0:00:12
   - -------------------------------------- 1.0/38.1 MB 3.3 MB/s eta 0:00:12
   - -------------------------------------- 1.2/38.1 MB 3.3 MB/s eta 0:00:12
   - -------------------------------------- 1.3/38.1 MB 3.4 MB/s eta 0:00:11
   - -----------

In [6]:
import numpy as np
import mediapipe as mp
import tensorflow as tf
import pyvirtualcam

# Load the MediaPipe hands model
mp_hands = mp.solutions.hands.Hands(
    max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5
)

# Load the TensorFlow model for gesture recognition
model = tf.keras.models.load_model('gesture_recognition_model.h5')

# Define a dictionary to map gestures to tools
gesture_to_tool = {
    'fist': 'hammer',
    'thumbs_up': 'power_tool',
    'open_palm': 'gripper',
    'pointing': 'laser_pointer'
}

# Initialize the PyVirtualCam object
pvcam = pyvirtualcam.VirtualCam(
    width=1920,
    height=1080,
    fps=30
)

# Capture frames from the camera
cap = cv2.VideoCapture(0)

while True:
    # Read the frame from the camera
    ret, frame = cap.read()

    # Convert the frame to RGB format
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect hands in the frame
    results = mp_hands.process(rgb_frame)

    # Check if any hands were detected
    if results.multi_hand_landmarks:
        # Get the landmarks of the first hand
        hand_landmarks = results.multi_hand_landmarks[0]

        # Convert the landmarks to a NumPy array
        landmarks_array = np.array([
            [landmark.x, landmark.y, landmark.z] for landmark in hand_landmarks.landmark
        ])

        # Normalize the landmarks
        landmarks_array = landmarks_array / np.linalg.norm(landmarks_array)

        # Predict the gesture
        gesture_prediction = model.predict(landmarks_array.reshape(1, -1))[0]

        # Get the corresponding tool
        tool = gesture_to_tool[gesture_prediction]

        # Display the tool on the virtual camera
        pvcam.display_array(frame)
        pvcam.display_overlay_text(tool, (50, 50), color=(0, 255, 0), size=30)

    # Check if the user wants to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Close the camera and the virtual camera
cap.release()
pvcam.stop()

OSError: No file or directory found at gesture_recognition_model.h5